## Import packages

In [1]:
import pandas as pd
from CFModel import CFModel

/home/dislab/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Define constants


In [2]:
RATINGS_CSV_FILE = './input/rating_result.csv'
USERS_CSV_FILE = './input/user_result.csv'
RECIPE_CSV_FILE = './input/recipe_result.csv'
MODEL_WEIGHTS_FILE = 'recipe_weights.h5'
K_FACTORS = 120
TEST_USER = 10

## Load Recipe data

In [3]:
ratings = pd.read_csv(RATINGS_CSV_FILE, sep=',', encoding='utf-8', usecols=['userid', 'recipeid', 'rating'])
max_userid = ratings['userid'].drop_duplicates().max() + 1
max_recipeid = ratings['recipeid'].drop_duplicates().max() + 1
print(len(ratings), 'ratings loaded.')

(1000000, 'ratings loaded.')


In [4]:
users = pd.read_csv(USERS_CSV_FILE, sep=',', encoding='utf-8', usecols=['userid', 'nickname'])
print(len(users), 'descriptions of', max_userid, 'users loaded.')

(75061, 'descriptions of', 12375, 'users loaded.')


In [5]:
recipes = pd.read_csv(RECIPE_CSV_FILE, sep=',', encoding='utf-8', usecols=['recipeid'])#, 'title', 'categories'])
print(len(recipes), 'descriptions of', max_recipeid, 'recipes loaded.')

(5101, 'descriptions of', 5023, 'recipes loaded.')


## Make recommendations for a given user

In [6]:
trained_model = CFModel(max_userid, max_recipeid, K_FACTORS)

CFModel.py:25: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  self.add(Merge([P, Q], mode='dot', dot_axes=1))


In [7]:
trained_model.load_weights(MODEL_WEIGHTS_FILE)

In [8]:
users[users['userid'] == TEST_USER]

,userid,nickname
10,10,dreamlove1


In [9]:
def predict_rating(userid, recipeid):
    return trained_model.rate(userid, recipeid)

In [10]:
user_ratings = ratings[ratings['userid'] == TEST_USER][['userid', 'recipeid', 'rating']]

In [11]:
user_ratings['prediction'] = user_ratings.apply(lambda x: predict_rating(TEST_USER, x['recipeid']), axis=1)
user_ratings.sort_values(by='rating', 
                         ascending=False).merge(recipes, 
                                                on='recipeid', 
                                                how='inner',
                                                suffixes=['_u', '_m']
                                               ).head(10)

,userid,recipeid,rating,prediction
0,10,331,5,5.105783
1,10,159,5,5.134003
2,10,9,5,4.304326
3,10,20,5,5.101132
4,10,44,5,5.094398
5,10,70,5,5.094523
6,10,107,5,5.071193
7,10,129,5,5.099595
8,10,133,5,5.087827
9,10,135,5,5.046216


In [14]:
recommendations = ratings[ratings['recipeid'].isin(user_ratings['recipeid']) == False][['recipeid']].drop_duplicates()
recommendations['prediction'] = recommendations.apply(lambda x: predict_rating(TEST_USER, x['recipeid']), axis=1)
recommendations.sort_values(by='prediction',
                          ascending=False).merge(recipes,
                                                 on='recipeid',
                                                 how='inner',
                                                 suffixes=['_u', '_m']).head(10)

,recipeid,prediction
0,503,5.157289
1,619,5.155896
2,3337,5.155661
3,2088,5.153944
4,498,5.153167
5,1217,5.150778
6,109,5.146888
7,997,5.146050
8,1704,5.145755
9,693,5.145190
